# AutoNeural - NPU-Native Multimodal Model for Automotive Cockpits

This notebook demonstrates the 4 core use cases of AutoNeural:

1. **In-Cabin Detection** - Detect risky behaviors and situations
2. **Out-Cabin Awareness** - Read parking signs and surrounding context
3. **HMI & Dashboard Understanding** - Understand dashboards and warning lights
4. **Visual + Conversational Agentic Tasks** - Ground natural language in visual context

> **Note:** AutoNeural is optimized for Qualcomm NPUs and requires proper authentication setup.


## Prerequisites

### 1. Install the correct Python version

If you prefer, we also offer a video tutorial for the installation. Check it out [here](https://www.youtube.com/watch?v=ziXKPRX0Ufo).

NexaAI requires **Python 3.11 – 3.13 (ARM64 build)** on Windows ARM.
Please **download and install the official ARM64 Python** from the [python-3.11.1-arm64.exe](https://www.python.org/ftp/python/3.11.1/python-3.11.1-arm64.exe). Make sure you read the instructions below carefully before proceeding.

> ❗ **IMPORTANT**: Make sure you select "Add python.exe to PATH" on the first screen of the installation wizard.

> 🛑 Make sure you restart the terminal or your IDE after installation.

> ⚠️ Do **not** use Conda or x86 builds — they are incompatible with native ARM64 binaries. If you are in a conda environment, run `conda deactivate` first.

Verify the installation:

In case your environment path gets overriden by some environment manager, we recommend you to run the following commands to restore PATH variable from system settings.
```powershell
$systemPath = [Environment]::GetEnvironmentVariable('Path', 'Machine')
$userPath   = [Environment]::GetEnvironmentVariable('Path', 'User')
$env:Path   = "$userPath;$systemPath"
```

Then verify your python executable has the correct architecture and version (3.11 - 3.13)
```sh
python -c "import sys, platform; print(f'Python version: {sys.version}')"
```

Your output should look like:

> Python version: 3.11.0 (main, Oct 24 2022, 18:15:22) [MSC v.1933 64 bit (ARM64)]

Expected output must contain version `3.11.0` and architecture `ARM64`.

If it does show `AMD64` or incorrect version, try the following:

- (If you have conda installed) Run `conda deactivate` to deactivate the current conda environment.
- (If your `python` executable points to the x86 version) You may need to make the ARM64 Python come before the x86 Python in your PATH.
  - Hit the `Win` key, and type `env`, and hit Enter to select `Edit the system environment variables` setting.
  - Click on `Environment Variables...` button.
  - Select `Path` and click `Edit...`.
  - Find your ARM64 Python installation path, and move it to the top of the list.
  - Hit `OK` for several times to close all the dialogs and save the changes.
- (If you forgot to select "Add python.exe to PATH" on the first screen of the installation wizard)
  - Run the installation wizard again, follow the instructions to remove the current installation, and then reinstall from the Wizard. Make sure to select "Add python.exe to PATH" this time.

---

### 2. Create and activate a virtual environment

`cd` to the current project root directory `cd path/to/nexa-sdk`.

```sh
python -m venv nexaai-env
nexaai-env\Scripts\activate
```

---

### 3. Install the NexaAI SDK

```bash
pip install nexaai
```

---

### 4. Select the venv as your Jupyter Notebook kernel

- Depending the editor you are using, the way to change kernel might be different. For Cursor / VS Code, they are located at the top right corner of you code window.
- Look for and select the `nexaai-env`, or the custom virtual environment you have created. The kernel should automatically reload in most IDEs.

## Authentication Setup

Before running any examples, you need to set up your NexaAI authentication token.

### Set Token in Code

Replace `"YOUR_NEXA_TOKEN_HERE"` with your actual NexaAI token from [https://sdk.nexa.ai/](https://sdk.nexa.ai/):


In [ ]:
import os

# Replace "YOUR_NEXA_TOKEN_HERE" with your actual token from https://sdk.nexa.ai/
os.environ["NEXA_TOKEN"] = "YOUR_NEXA_TOKEN_HERE"
# Suppress HF warnings
os.environ["HF_HUB_VERBOSITY"] = "error"

assert os.environ.get("NEXA_TOKEN", "").startswith(
    "key/"), "ERROR: NEXA_TOKEN must start with 'key/'. Please check your token."

## Setup and Model Loading


In [ ]:
from pathlib import Path
from IPython.display import Image, display
from nexaai.vlm import VLM, GenerationConfig
from nexaai.common import ModelConfig, MultiModalMessage

# Configure paths
script_dir = Path(".").resolve()
assets_dir = script_dir / "images"

print("Loading AutoNeural model...")
vlm = VLM.from_(name_or_path="NexaAI/AutoNeural", m_cfg= ModelConfig(), plugin_id="npu")
print("✅ Model loaded successfully!")

def run_example(vlm: VLM, image_path: str | Path, prompt: str, max_tokens: int = 512):
    """Run a single VLM inference example."""
    image_path_str = str(image_path)
    
    conversation: list[MultiModalMessage] = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": "You are a helpful automotive assistant.",
                    "url": None,
                    "path": None,
                }
            ],
        },
        {
            "role": "user",
            "content": [
                {"type": "image", "path": image_path_str, "text": None, "url": None},
                {"type": "text", "text": prompt, "url": None, "path": None},
            ],
        },
    ]

    formatted_prompt = vlm.apply_chat_template(conversation)
    
    response = ""
    for token in vlm.generate_stream(
        formatted_prompt,
        g_cfg=GenerationConfig(max_tokens=max_tokens, image_paths=[image_path_str]),
    ):
        print(token, end="", flush=True)
        response += token
    
    return response


## Example 1: In-Cabin Detection

Detect risky behaviors and situations in the cabin, such as driver distraction, drowsiness, children seated unsafely, or passengers blocking visibility.


In [ ]:
image_path = assets_dir / "in-cabin.png"
display(Image(str(image_path), width=600))

prompt = "Any safety risks for my child right now? Explain it in great detail."

print("Prompt:", prompt)
print("\nResponse:")
response = run_example(vlm, image_path, prompt)

## Example 2: Out-Cabin Awareness

Read real-world parking signs and surrounding context to answer "Can I park here?" type questions.


In [ ]:
image_path = assets_dir / "out-cabin.png"
display(Image(str(image_path), width=600))

prompt = "What are the parking rules here? Provide detailed explanation"

print("Prompt:", prompt)
print("\nResponse:")
response = run_example(vlm, image_path, prompt)


## Example 3: HMI & Dashboard Understanding

Understand dashboards, warning lights, and infotainment UIs and explain them in natural language.


In [ ]:
image_path = assets_dir / "dashboard.png"
display(Image(str(image_path), width=600))

prompt = "How do I address this issue? what does the issue mean?"

print("Prompt:", prompt)
print("\nResponse:")
response = run_example(vlm, image_path, prompt)


## Example 4: Visual + Conversational Agentic Tasks

Ground natural language requests in visual context and emit structured outputs that your app can consume.


In [ ]:
image_path = assets_dir / "UI.png"
display(Image(str(image_path), width=600))

prompt = "Can you navigate me to the event?"

print("Prompt:", prompt)
print("\nResponse:")
response = run_example(vlm, image_path, prompt)


## Summary

All examples have been completed! AutoNeural demonstrates its capabilities across:

- ✅ **In-Cabin Safety** - Real-time detection of risky behaviors
- ✅ **Out-Cabin Awareness** - Understanding parking signs and context
- ✅ **Dashboard Intelligence** - Interpreting vehicle status and UI
- ✅ **Agentic Tasks** - Structured visual understanding for applications

For more information, visit the [AutoNeural model page](https://huggingface.co/NexaAI/AutoNeural) or check the [README](./README.md).
